# Code to migrate the data files in the archive to JSON

We want all the data in a modern format.

# WWII Data

- allied.xml - list of allied planes
- axis.xml - lise of axis planes
- holtgrewe.xml - This is the primary data file.
- index.xml - Looks like some kind of mapping file???

# Holtgrewe Models

The main location for data (in the WWII collection) is holtgrewe.xml.  It
consists of a series of <model> tags.  Here is an example:

```xml
<model index="1" holtgrewe="001" affiliation="1" cos="Great Britain" com="Great Britain" logo="grb">
  <img name="001"></img>
  <title><![CDATA[Miles M.9A Master Mk I]]></title>
  <type type="Advanced Trainer"></type>
  <firstYR>1941</firstYR>
  <displayedAs>
    <![CDATA[This model represents an aircraft of the Royal Air Force No. ...]]>
  </displayedAs>
  <history>
    <![CDATA[As monoplane fighters began to equip Royal Air Force units, ...]]>
  </history>
  <tech>
    <powerplant>
      <![CDATA[One Rolls-Royce Kestrel XXX, liquid-cooled inline piston engine of 715 hp (533 kW)]]>
    </powerplant>
    <performance>
      <maxspeed>226 mph at 15,000 feet (364 km/h at 4,570 m)</maxspeed>
      <ceiling>28,000 feet (8,500 m)</ceiling>
      <range>500 miles (805 km)</range>
    </performance>
    <weight>4,160 pounds (1,886 kg)</weight>
    <maxweight>5,352 pounds (2,428 kg)</maxweight>
    <dimensions span="38 feet 10 inches (11.85 m)" lengths="30 feet 5 inches (9.27 m)" height="9 feet 3 inches (2.82 m)">
    </dimensions>
    <armament><![CDATA[Provisions for one fixed, forward-firing Vickers 0.303 i...]]></armament>
    <crew num="2 [one student, one instructor]"></crew>
    <comments>
      <![CDATA[Model Builder's Comments: This Master has the typical... "]]>
    </comments>
  </tech>
</model>
```

We desired to convert these models into JSON format for use in a web application.
This could be encoded as follows (editing some fields for clarity):

```json
{
  modelNumber: "001",
  countryOfService: "Great Britain",
  countryOfManufacture: "Great Britain",
  logo: "grb",
  title: "Miles M.9A Master Mk I",
  type: "Advanced Trainer",
  firstYear: 1941,
  displayedAs: "This model represents an aircraft of the Royal Air Force No. ... ",
  history: "As monoplane fighters began to equip Royal Air Force units, ...",
  tech: {
    powerplant: "One Rolls-Royce Kestrel XXX, liquid-cooled inline piston engine of 715 hp (533 kW)",
    performance: {
      maxSpeed: "226 mph at 15,000 feet (364 km/h at 4,570 m)",
      ceiling: "28,000 feet (8,500 m)"
      range: "500 miles (805 km)"
    },
    weight: "4,160 pounds (1,886 kg)",
    maxWeight: "5,352 pounds (2,428 kg)",
    dimensions: {
      span: "38 feet 10 inches (11.85 m)",
      length: "30 feet 5 inches (9.27 m)",
      height: "9 feet 3 inches (2.82 m)"
    }
    armament: "Provisions for one fixed, forward-firing Vickers 0.303 i... ",
    crewSize: "2 [one student, one instructor]",
    comments: "Model Builder's Comments: This Master has the typical... "
  }
}
```

We could make this data more generally useful, by removing formatting from
data fields, allowing the display software to convert units in a standard
way.  As it appears that the standard units are English, I would adopt that
for data values (feet (decimal), miles, pounds, mph).  This would further
convert this model to:

```json
{
  modelNumber: "001",
  countryOfService: "Great Britain",
  countryOfManufacture: "Great Britain",
  logo: "grb",
  title: "Miles M.9A Master Mk I",
  type: "Advanced Trainer",
  firstYear: 1941,
  displayedAs: "This model represents an aircraft of the Royal Air Force No. ... ",
  history: "As monoplane fighters began to equip Royal Air Force units, ...",
  tech: {
    powerplant: {
      engines: 1,
      hp: 715,
      description: "Rolls-Royce Kestrel XXX, liquid-cooled inline piston engine",
    },
    performance: {
      maxSpeed: {
        speed: 226,
        altitude: 15000
       }
      ceiling: 28000,
      range: 500,
    },
    weight: 4160,
    maxWeight: 5352,
    dimensions: {
      span: 10.83,
      length: 30.42,
      height: 9.25,
    }
    armament: "Provisions for one fixed, forward-firing Vickers 0.303 i... ",
    crew: {
      size: 2,
      makeup: "one student, one instructor"
    },
    comments: "Model Builder's Comments: This Master has the typical... "
  }
}
```

In [34]:
!ls -al ../archive/wwii/data

total 1156
drwxrwxr-x 2 mckoss mckoss    4096 Jan 26 16:00 .
drwxrwxr-x 4 mckoss mckoss    4096 Jan 26 17:18 ..
-rw-rw-r-- 1 mckoss mckoss   14879 Jan 26 15:59 allied.xml
-rw-rw-r-- 1 mckoss mckoss   13466 Jan 26 15:59 axis.xml
-rw-rw-r-- 1 mckoss mckoss 1133333 Jan 26 15:59 holtgrewe.xml
-rw-rw-r-- 1 mckoss mckoss    6213 Jan 26 15:59 index.xml


In [ ]:
!pip install xmltodict

In [2]:
DATA_DIR = '../archive/wwii/data'

In [4]:
import json
import xmltodict

with open(f"{DATA_DIR}/holtgrewe.xml", 'r') as f:
    data = f.readlines()

# Parser needs a single top-level element - so we'll wrap the data in a root element
data.insert(1, '<root>\n')
data.append('</root>\n')

root = xmltodict.parse(''.join(data))
print(json.dumps(root['root']['model'][0], indent=2))

{
  "@index": "1",
  "@holtgrewe": "001",
  "@affiliation": "1",
  "@cos": "Great Britain",
  "@com": "Great Britain",
  "@logo": "grb",
  "img": {
    "@name": "001"
  },
  "title": "Miles M.9A Master Mk I",
  "type": {
    "@type": "Advanced Trainer"
  },
  "firstYR": "1941",
  "displayedAs": "This model represents an aircraft of the Royal Air Force No. 8 Flying Training School, Ternhill, England, April 1940.",
  "history": "As monoplane fighters began to equip Royal Air Force units, it became obvious that an advanced trainer was needed to transition new young pilots from their basic elementary trainers to the high performance fighters they would be flying. To fulfill this need, the Air Ministry issued a specification for a high performance trainer. George Miles designed the \"Master\" in response. The new trainer was of smooth aeronautical design and was built of wood. Miles' first prototype, which first flew on June 3, 1937, was rejected by the Air Ministry, but Miles continued on 